<a href="https://colab.research.google.com/github/potasali/Machine-Learning/blob/master/Programming_Assignment_5_k_Nearest_Neighbors/Programming_Assignment_5_k_Nearest_Neighbors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programming Assignment 5: k Nearest Neighbors


## Problem
The purpose of this assignment is to get you familiar with k nearest neighbor classification.
You are given with Iris Data Set that contains information of three different species of iris
flower. Your task is to implement k nearest classifier and use it for predicting the flower
species given measurements of iris flowers.



## 1. Dataset Loading
The data set contains 150 instances with 5 attributes (4 input variables and 1 output class).
Attribute Information:
- Sepal length in cm
- Sepal width in cm
- Petal length in cm
- Petal width in cm
- Class
 - Iris Setosa
 - Iris Versicolour
 - Iris Virginica 

The data set has been divided into two sets.
- train.csv: 135 instances (45 per class)
- test.csv: 15 instances (5 per class)

In [0]:
# Importing Libraries
import os
import glob
import re
import numpy as np
from matplotlib import pyplot
import math 
import pandas as pd
import string
import heapq
from statistics import mode
from nltk.corpus import stopwords

In [0]:
# Loading the train and test datasets
column_names = ["Sepal Length", "Sepal Width", "Petal Length", "Petal Width", "Class"]
train = pd.read_csv (r'train.csv', names = column_names)
test = pd.read_csv (r'test.csv', names = column_names)

In [0]:
# Changing datatypes of the dataframe to float
train["Sepal Length"] = train["Sepal Length"].astype('float64')
train["Sepal Width"] = train["Sepal Width"].astype('float64')
train["Petal Length"] = train["Petal Length"].astype('float64')
train["Petal Width"] = train["Petal Width"].astype('float64')
test["Sepal Length"] = test["Sepal Length"].astype('float64')
test["Sepal Width"] = test["Sepal Width"].astype('float64')
test["Petal Length"] = test["Petal Length"].astype('float64')
test["Petal Width"] = test["Petal Width"].astype('float64')

In [0]:
def labelEncoding(data, arr):
    for label in data:

        # Labeling Iris-setosa as 0
        if label == 'Iris-setosa':
            arr.append(0)

        # Labeling Iris-versicolor as 1
        elif label == 'Iris-versicolor':
            arr.append(1)

        # Labeling Iris-virginica as 2
        elif label == 'Iris-virginica':
            arr.append(2)
            
    arr = np.asarray(arr)
    
test_y = []
labelEncoding(test["Class"], test_y)

## 2. KNN Implementation

In [0]:
# Finding the euclidian distance between two vectors P and Q
def euclidianDistance(P, Q, n):

    # Initializing the distance
    distance = 0

    # Iterating through elements in both vectors and finding Euclidian distance
    for i in range(n):
        distance += np.square(P[i] - Q[i])
    distance = np.sqrt(distance)

    return distance

def findingDistances(train, test):
    distances = []

    # Iterating through each test sample
    for i in range(len(test)):
        test_i = test.iloc[i]

        # Initializing a distance array
        test_i_distances = []

        # Iterating through each train sample
        for j in range(len(train)):
            train_i = train.iloc[j]

            # Finding the euclidian distance between the current test sample with each train sample
            # Appending the euclidian distance along with the train label as a tuple in the distance array for each test pint
            test_i_distances.append((euclidianDistance(test_i, train_i, 4), train_i[4]))

        # Appending the array of tuples in the distance array 
        distances.append(test_i_distances)

    # Returns a 2D array of tuples containing distance between each test and train point along with the train label
    return distances

# Sort the distance array     
def sortDistances(distances):
    sortedArray = []

    # Iterate through each test point distances to the train point
    for i in range(len(distances)):

        # Sort the tuple array according to the distance
        sortedArray.append(sorted(distances[i], key = lambda x: x[0]))

    #  Returns a sorted 2D array of tuples containing distance between each test and train point along with the train label
    return sortedArray
    
# prediction function for the KNN algorithm
def KNN(distances, k):
    
    # Initializes the label array
    labels = []

    # Iterate through each test point distances to the train point
    for i in range(len(distances)):

        # Initialize the k nearest neighbor array
        nearest_k = []

        # Taking out the k first values from the distance array and store ther labels
        for j in range(k):
            this_label = distances[i][j][1]
            nearest_k.append(this_label)

        # Find the mode value of the k neighbors
        mode_label = mode(nearest_k)
        labels.append(mode_label)
    
    # Returns predicted labels
    return labels

def Algo(train, test, k):

    # Find eucladian distances of the test samples from each train sample
    distances = findingDistances(train, test)

    # Sorting the distance array
    distances = sortDistances(distances)

    # Finding the predicted value for the given value of k
    labels = KNN(distances, k)
    labels
    predicts = []

    # Label encode the predicted values
    labelEncoding(labels, predicts)

    # Return label encoded predicted values
    return predicts

In [0]:
k = []
k.append(0)

# Finding k for values 1 to 5
for i in range(1, 6):
    k.append(Algo(train, test, i))

## 3. Evaluation

In [0]:
def eval(prediction, expected):
    
    # Initializing the confusion matrix along with all the outcomes
    tp = [0,0,0]
    tn = [0,0,0]
    fp = [0,0,0]
    fn = [0,0,0]
    CM = np.full((3,3), 0)

    # Updating the confusion matrix
    for E, P in zip(expected, prediction): 
        CM[P][E] += 1
        
    # Updating values for the true-positives, true-negatives, false-positives, false-negatives
    for i in range(3):
        tp[i] = CM[i][i]
        fp[i] = (CM[:, i]).sum() - tp[i]
        fn[i] = (CM[i, :]).sum() - tp[i]
        tn[i] = (CM.sum()) - tp[i] - fp[i] - fn[i]
    
    # Displaying the Confusion Matrix
    CM = pd.DataFrame(data=CM)
    CM.columns = ['Iris Setosa', 'Iris Versicolour', 'Iris Virginica']
    CM.rename(index={0:'Iris Setosa',1:'Iris Versicolour',2:'Iris Virginica'}, inplace=True)
    print("Confusion Matrix\n\n", CM)
    
    # Finding the pooled confusion matrix
    poolCM = np.array([[sum(tp), sum(fp)],[sum(fn), sum(tn)]])
    print("\nPooled: \n", poolCM)
    
    # Micro-Averaging Precision 
    print("\nMicro-Average Precision: ", (poolCM[0][0]/(poolCM[0][1]+poolCM[0][0])))
    
    # Micro-Averaging Recall 
    print("Micro-Average Recall: ", (poolCM[0][0]/(poolCM[1][0]+poolCM[0][0])))

    # Micro-Averaging Accuracy 
    print("Micro-Average Accuracy: ", ((poolCM[0][0]+sum(tn))/(poolCM[1][0]+poolCM[0][0]+poolCM[0][1]+poolCM[1][1])))

    # Micro-Averaging F1 score 
    miP = (poolCM[0][0]/(poolCM[0][1]+poolCM[0][0]))
    miR = (poolCM[0][0]/(poolCM[1][0]+poolCM[0][0]))
    f1_score = (2 * miP * miR) / (miP + miR)
    print("Micro-Average F1-Score: ", f1_score)
    
    # Finding Precison, Accuracy and Recall for each class
    maP = []
    maR = []
    maA = []
    for i in range(3):
        maP.append(tp[i] / (tp[i] + fp[i]))
        maR.append(tp[i] / (tp[i] + fn[i]))
        maA.append((tp[i] + tn[i]) / (tp[i] + tn[i] + fp[i] + fn[i]))

    # Macro-Average Precision
    _maP = (sum(maP))/3
    print("\nMacro-Average Precision: ", _maP)

    # Macro-Average Recall
    _maR = (sum(maR))/3
    print("Macro-Average Recall: ", _maR)

    # Macro-Average Accuracy
    _maA = (sum(maA))/3
    print("Macro-Average Accuracy: ", _maA)

    # Macro-Average F1 score
    f1_score = (2 * _maP * _maR) / (_maP + _maR

# Evaluation for k = 1
print("k = 1\n")
eval(k[1], test_y)  


k = 1

Confusion Matrix

                   Iris Setosa  Iris Versicolour  Iris Virginica
Iris Setosa                 5                 0               0
Iris Versicolour            0                 5               0
Iris Virginica              0                 0               5

Pooled: 
 [[15  0]
 [ 0 30]]

Micro-Average Precision:  1.0
Micro-Average Recall:  1.0
Micro-Average Accuracy:  1.0
Micro-Average F1-Score:  1.0

Macro-Average Precision:  1.0
Macro-Average Recall:  1.0
Macro-Average Accuracy:  1.0
Macro-Average F1-Score:  1.0


In [0]:
# Evaluation for k = 3
print("k = 3\n")
eval(k[3], test_y)  

k = 3

Confusion Matrix

                   Iris Setosa  Iris Versicolour  Iris Virginica
Iris Setosa                 5                 0               0
Iris Versicolour            0                 5               0
Iris Virginica              0                 0               5

Pooled: 
 [[15  0]
 [ 0 30]]

Micro-Average Precision:  1.0
Micro-Average Recall:  1.0
Micro-Average Accuracy:  1.0
Micro-Average F1-Score:  1.0

Macro-Average Precision:  1.0
Macro-Average Recall:  1.0
Macro-Average Accuracy:  1.0
Macro-Average F1-Score:  1.0


In [0]:
# Evaluation for k = 5
print("k = 5\n")
eval(k[5], test_y)  

k = 5

Confusion Matrix

                   Iris Setosa  Iris Versicolour  Iris Virginica
Iris Setosa                 5                 0               0
Iris Versicolour            0                 5               0
Iris Virginica              0                 0               5

Pooled: 
 [[15  0]
 [ 0 30]]

Micro-Average Precision:  1.0
Micro-Average Recall:  1.0
Micro-Average Accuracy:  1.0
Micro-Average F1-Score:  1.0

Macro-Average Precision:  1.0
Macro-Average Recall:  1.0
Macro-Average Accuracy:  1.0
Macro-Average F1-Score:  1.0
